In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import psutil
import pandas as pd
import numpy as np
import json
import joblib
from datetime import datetime

In [3]:
# CARGAR MODELO Y PREPROCESADORES
modelo = joblib.load('/content/drive/MyDrive/TFG/Deteccion de malware/modelo_deteccion_malware.pkl')
scaler = joblib.load('/content/drive/MyDrive/TFG/Deteccion de malware/scaler.pkl')
label_encoder = joblib.load('/content/drive/MyDrive/TFG/Deteccion de malware/label_encoder.pkl')

In [4]:
# RECOGER PROCESOS DEL SISTEMA
procesos = []
for proc in psutil.process_iter(['pid', 'ppid', 'name', 'num_threads', 'username']):
    try:
        info = proc.info
        procesos.append({
            'pslist.nproc': 1,
            'pslist.nppid': info['ppid'],
            'pslist.avg_threads': info['num_threads'],
            'pslist.nprocs64bit': 1,  # valor simbólico, no disponible
            'pslist.avg_handlers': 20,  # estimado arbitrario
            'dlllist.ndlls': 0,  # no disponible
            'dlllist.avg_dlls_per_proc': 0,
            'handles.nhandles': 100,  # estimación fija
            'handles.avg_handles_per_proc': 100,
            'malfind.ninjections': 0,
            'malfind.commitCharge': 0,
            'malfind.protection': 0,
            'malfind.uniqueInjections': 0,
            'nombre_proceso': info['name']
        })
    except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
        continue

df_nuevo = pd.DataFrame(procesos)

In [5]:
# AJUSTAR COLUMNAS AL MODELO
columnas_esperadas = scaler.feature_names_in_
for col in columnas_esperadas:
    if col not in df_nuevo.columns:
        df_nuevo[col] = 0  # Añadimos columnas faltantes con 0

df_nuevo = df_nuevo[columnas_esperadas]

# Escalado
df_nuevo_scaled = scaler.transform(df_nuevo)

In [6]:
# PREDICCIÓN
predicciones = modelo.predict(df_nuevo_scaled)
probas = modelo.predict_proba(df_nuevo_scaled)

# Convertimos predicciones
resultado = []
for i in range(len(predicciones)):
    resultado.append({
        "nombre": procesos[i]["nombre_proceso"],
        "malware": bool(predicciones[i]),
        "probabilidad": round(float(probas[i][1]), 4)
    })

In [7]:
# GUARDAR EN JSON
salida = {
    "ultimo_analisis": datetime.now().strftime("%Y-%m-%d %H:%M"),
    "procesos_analizados": len(resultado),
    "resultados": resultado
}

# Guardamos en Drive (puedes cambiar ruta)
with open('/content/drive/MyDrive/TFG/analisis_sistema.json', 'w') as f:
    json.dump(salida, f, indent=4)

print("Análisis guardado en JSON")

Análisis guardado en JSON
